In [1]:
import cv2
import numpy as np
import imutils
from collections import deque

In [2]:
#after testing hsv 
greenLower = (29, 86, 6)
greenUpper = (64, 255, 255)
# greenLower = (41, 30, 74)
# greenUpper = (86, 144, 255)
pts = deque(maxlen=20)

In [51]:
video = cv2.VideoCapture("3.mov")

fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
out = cv2.VideoWriter('outputtt.avi',fourcc, 25.0,(1000,562),True)

xmin=1000
final=None
xxx="No Goal"
while(video.isOpened()):
    _, frame = video.read()
    if frame is None:
        break
    frame = imutils.rotate(frame, 180)
    frame = imutils.resize(frame, width=1000)
    
    
    frame2=frame
    gray = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)                                            #zzzzzzzzzzzzzzzzz
    blur_gray = cv2.GaussianBlur(gray,(5,5),0)
    edges = cv2.Canny(blur_gray, 50, 150)
    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 15  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 50  # minimum number of pixels making up a line
    max_line_gap = 20  # maximum gap in pixels between connectable line segments
    line_image = np.copy(frame2) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)

    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
        
    # Draw the lines on the  image
    lines_edges = cv2.addWeighted(frame2, 0.8, line_image, 1, 0)
    
    cv2.imshow("x",frame2)
    
    
    
    
    
    
    blurred = cv2.GaussianBlur(frame, (11, 11), 0)
#     cv2.imshow("blurred",blurred)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
#     cv2.imshow("hsv",hsv)
    mask = cv2.inRange(hsv, greenLower, greenUpper)
    kernel=np.ones((5,5),np.uint8)
#     mask = cv2.erode(mask, kernel, iterations=2)
#     mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel)
    mask=cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
    mask = cv2.dilate(mask, kernel, iterations=2)
    mask = cv2.erode(mask, kernel, iterations=2)
    res=cv2.bitwise_and(frame,frame,mask=mask)
    cnts,_=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2:]
#     cv2.imshow("mask",mask)
    
#     cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     cnts = imutils.grab_contours(cnts)
    center = None
    
    # only proceed if at least one contour was found
    if len(cnts) > 0:
        # find the largest contour in the mask, then use
        # it to compute the minimum enclosing circle and
        # centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        
        if x<xmin:
            xmin=x
            final=frame
            
        #M = cv2.moments(c)
        #center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        center = (int(x), int(y))
        
        # only proceed if the radius meets a minimum size
        if radius > 10:
            # draw the circle and centroid on the frame,
            # then update the list of tracked points
            cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
            cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)
            if x+radius > 650:
                xxx="Goal"
    
    pts.appendleft(center)
    for i in range (1,len(pts)):
        if pts[i-1]is None or pts[i] is None:
            continue
        thick = int(np.sqrt(len(pts) / float(i + 1)) * 2.5)
        cv2.line(frame, pts[i-1],pts[i],(0,0,225),thick)
    #print(x)
    #print(radius)


    #frame = imutils.rotate(frame, 180)
    if (xxx=="No Goal"):
        cv2.putText(frame, "No Goal", (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 6)
    else:
        cv2.putText(frame, "Goal", (10, 75), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 255, 0), 6)
    cv2.imshow("Frame", frame)
    #(h, w) = frame.shape[:2]
    #print(h)
    #print(w)
    out.write(frame)
    key = cv2.waitKey(1) & 0xFF
    
    if key == 27:
        break

if final is not None:
    cv2.imshow("aa",final)
    cv2.waitKey()

video.release()
out.release()
cv2.destroyAllWindows()

In [3]:
video = cv2.VideoCapture("3.mov")

xmin2=1000
while(video.isOpened()):
    _, frame = video.read()
    if frame is None:
        break
    frame = imutils.rotate(frame, 180)
    frame = imutils.resize(frame, width=1000)
    (h, w) = frame.shape[:2]
    
    
    frame2=frame.copy()
    hsv = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 212])
    upper_white = np.array([131, 255, 255])
#     low_yellow = np.array([80 ,41  ,0])
#     up_yellow = np.array([113 ,255 ,255])
    mask = cv2.inRange(hsv, low_yellow, up_yellow)
    kernel_erode = np.ones((4,4), np.uint8)
    eroded_mask = cv2.erode(mask, kernel_erode, iterations=1)
    kernel_dilate = np.ones((6,6),np.uint8)
    dilated_mask = cv2.dilate(eroded_mask, kernel_dilate, iterations=1)
    cv2.imshow("dilated_mask",dilated_mask)
    contours, hierarchy = cv2.findContours(dilated_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort by area (keep only the biggest one)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]
    if len(contours) > 0:
        cv2.drawContours(frame2, [contours[0]], -1, (0, 255, 0), 5)
        for i in range (0,len(contours[0])):
#             print(contours[0][i][0][0])
            if contours[0][i][0][0] < xmin2:
                xmin2=contours[0][i][0][0]
        cv2.line(frame2, (xmin2,0),(xmin2,h),(0,0,225),5)

    
    
    
    cv2.imshow("frame2",frame2)
    
    
    
    
    
    
    
    
    
    
    
#     kernel= np.ones((5,5),np.uint8)
#     edges = cv2.morphologyEx(edges,cv2.MORPH_CLOSE,kernel)
#     edges = cv2.dilate(edges, kernel, iterations=1)
#     edges = cv2.morphologyEx(edges,cv2.MORPH_CLOSE,kernel)

#     lines = cv2.HoughLinesP(edges, 1, np.pi/180, 200, maxLineGap=100)
#     print(lines.shape)
#     if lines is not None:
#         for line in lines:
#             x1, y1, x2, y2 = line[0]
#             cv2.line(frame2, (x1, y1), (x2, y2), (0, 255, 0), 5)
#             break
    
#     cv2.imshow("frame2", frame2)
#     cv2.imshow("edges", edges)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
#     blur_gray = cv2.GaussianBlur(gray,(5,5),0)
#     edges = cv2.Canny(blur_gray, 75 , 150)
# #     kernel=np.ones((5,5),np.uint8)
# #     edges = cv2.morphologyEx(edges,cv2.MORPH_CLOSE,kernel)
# #     edges = cv2.dilate(edges, kernel, iterations=1)
# #     edges = cv2.morphologyEx(edges,cv2.MORPH_CLOSE,kernel)
    
#     cv2.imshow("edges",edges)
#     threshold = 15  # minimum number of votes (intersections in Hough grid cell)
#     min_line_length = 200 # minimum number of pixels making up a line
#     max_line_gap = 200  # maximum gap in pixels between connectable line segments
#     line_image = np.copy(frame2) * 0  # creating a blank to draw lines on

#     # Run Hough on edge detected image
#     # Output "lines" is an array containing endpoints of detected line segments
#     lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold, np.array([]),
#                         min_line_length, max_line_gap)

#     for line in lines:
#         for x1,y1,x2,y2 in line:
#             cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
        
#     # Draw the lines on the  image
#     lines_edges = cv2.addWeighted(frame2, 0.8, line_image, 1, 0)
    
#     cv2.imshow("x",lines_edges)
    
    
    

    key = cv2.waitKey(1) & 0xFF
    
    if key == 27:
        break

video.release()
cv2.destroyAllWindows()

NameError: name 'low_yellow' is not defined

In [48]:
video.release()
# out.release()
cv2.destroyAllWindows()

In [23]:
video = cv2.VideoCapture("3.mov")

xmin2=1000
while(video.isOpened()):
    _, frame = video.read()
    if frame is None:
        break
    frame = imutils.rotate(frame, 180)
    frame = imutils.resize(frame, width=1000)
    (h, w) = frame.shape[:2]
    
    
    frame2=frame.copy()
    hsv = cv2.cvtColor(frame2, cv2.COLOR_BGR2HSV)
    lower_white = np.array([0, 0, 212])
    upper_white = np.array([131, 255, 255])
    low_yellow = np.array([80 ,41  ,0])
    up_yellow = np.array([113 ,255 ,255])
    mask = cv2.inRange(hsv, low_yellow, up_yellow)
    kernel_erode = np.ones((4,4), np.uint8)
    eroded_mask = cv2.erode(mask, kernel_erode, iterations=1)
    kernel_dilate = np.ones((6,6),np.uint8)
    dilated_mask = cv2.dilate(eroded_mask, kernel_dilate, iterations=1)
    cv2.imshow("dilated_mask",dilated_mask)
    
    
    contours, hierarchy = cv2.findContours(dilated_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort by area (keep only the biggest one)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]
    if len(contours) > 0:
        peri = cv2.arcLength(contours[0] , True)
        approx = cv2.approxPolyDP(contours[0], 0.01 * peri, True)
        # ensure that the approximated contour is "roughly" rectangular
        if len(approx) >= 4 and len(approx) <= 6:
            print(approx[0][0])
            point1=(approx[0][0][0],approx[0][0][1])
            point2=(approx[1][0][0],approx[1][0][1])
            cv2.line(frame2,point1,point2,(255,0,0),5)

    
    
    
    cv2.imshow("frame2",frame2)

    
    
    

    key = cv2.waitKey(1) & 0xFF
    
    if key == 27:
        break

video.release()
cv2.destroyAllWindows()

[510   0]
[510   0]
[510   0]
[511   0]
[512   0]
[513   0]
[514   0]
[513   0]
[513   0]
[513   0]
[512   0]
[513   0]
[513   0]
[514   0]
[514   0]
[514   0]
[514   0]
[514   0]
[513   0]
[513   0]
[514   0]
[514   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[514   0]
[514   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[514   0]
[514   0]
[513   0]
[514   0]
[514   0]
[514   0]
[514   0]
[514   0]
[514   0]
[514   0]
[514   0]
[514   0]
[514   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[513   0]
[514   0]
[513   0]
[513   0]
[514   0]
[513   0]
[513   0]
[514   0]
[514   0]
[514   0]
[514   0]
[514   0]
[513   0]


In [20]:
video.release()
cv2.destroyAllWindows()